In [ ]:
import arxiv
import openai
import requests
import base64
import time
import hashlib
import json
from datetime import datetime, timedelta, timezone
from IPython.display import display, Markdown, Image
from requests.exceptions import RequestException
from io import BytesIO

# Import secrets from separate file (not committed to git)
from secrets_config import (
    OPENAI_API_KEY,
    THREADS_USER_ID,
    THREADS_ACCESS_TOKEN,
    APP_SECRET,
    GITHUB_TOKEN,
    GITHUB_REPO,
    GITHUB_PAGES_SITE,
    FACEBOOK_ID,
    INSTAGRAM_ACCESS_TOKEN
)

# ============================================================================
# BLOG AUTOMATION CONFIGURATION - Modify these as needed
# ============================================================================

# arXiv paper settings
ARXIV_CATEGORIES = ["cs.AI", "cs.LG", "cs.CL", "cs.CV", "stat.ML"]
MAX_PAPERS_TO_PROCESS = 5       # Number of papers to fetch and potentially process
DAYS_BACK_TO_SEARCH = 7         # How many days back to search for new papers

# Content generation settings
BLOG_POST_LENGTH = "300-400"    # Target word count for blog posts
BLOG_POST_MAX_TOKENS = 500      # Max tokens for GPT response
BLOG_POST_TEMPERATURE = 0.7     # Creativity level (0.0-1.0)

# Threads post settings
THREADS_MAX_CHARS = 350         # Max characters for main Threads text
THREADS_HASHTAGS = "#AI #ArtificialIntelligence #MachineLearning #DataScience #Latest #Research #Arxiv #OpenAI"
THREADS_WAIT_TIME = 30          # Seconds to wait before publishing
THREADS_MAX_RETRIES = 3

# Image generation settings (DALL-E 3)
IMAGE_MODEL = "dall-e-3"        # "dall-e-2" or "dall-e-3"
IMAGE_QUALITY = "standard"      # "standard" or "hd" (hd costs more)
IMAGE_STYLE = "natural"         # "natural" or "vivid"
IMAGE_SIZE = "1024x1024"        # "1024x1024", "1792x1024", or "1024x1792"

# Processing settings
SKIP_EXISTING_POSTS = True      # Skip papers that already have blog posts
SAVE_IMAGES_TO_GITHUB = True    # Download and save images to prevent expiration
LINK_PREVIEW_WAIT = 30          # Seconds to wait for link preview generation
GITHUB_PAGES_IMAGE_WAIT = 60    # Seconds to wait for image deployment

# Testing/debugging settings
TEST_MODE = False               # Set to True to process only 1 paper for testing
VERBOSE_OUTPUT = True           # Show detailed processing information

# ============================================================================
# Auto-adjust settings based on test mode
# ============================================================================
if TEST_MODE:
    MAX_PAPERS_TO_PROCESS = 1
    print("🧪 TEST MODE ENABLED - Processing only 1 paper")

# Initialize the OpenAI client
client = openai.OpenAI(api_key=OPENAI_API_KEY)

def generate_harvard_reference(paper):
    authors = paper.authors
    
    if len(authors) == 1:
        author_str = authors[0].name
    elif len(authors) == 2:
        author_str = f"{authors[0].name} and {authors[1].name}"
    else:
        author_str = f"{authors[0].name} et al."
    
    year = paper.published.year
    title = paper.title
    
    reference = f"{author_str} ({year}) '{title}', arXiv preprint arXiv:{paper.get_short_id()}."
    
    return reference

def fetch_latest_papers(categories, max_results=12):  # Fixed: was max_results=1
    client_arxiv = arxiv.Client()  # Renamed to avoid confusion with OpenAI client
    last_week = datetime.now(timezone.utc) - timedelta(days=7)
    
    category_query = " OR ".join([f"cat:{cat}" for cat in categories])
    
    search = arxiv.Search(
        query = f"({category_query})",
        max_results = max_results,
        sort_by = arxiv.SortCriterion.SubmittedDate
    )
    
    results = list(client_arxiv.results(search))
    return [paper for paper in results if paper.published.replace(tzinfo=timezone.utc) > last_week]

def generate_blog_post(paper):
    authors = ', '.join([author.name for author in paper.authors])
    prompt = f"""Write an engaging blog post about the following scientific paper:

Title: {paper.title}
Authors: {authors}
Abstract: {paper.summary}

The blog post should:
1. Explain the main findings in simple terms
2. Discuss potential real-world implications
3. Be engaging and accessible to a general audience
4. Be around {BLOG_POST_LENGTH} words long

Blog Post:"""

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that writes engaging blog posts about scientific papers."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=BLOG_POST_MAX_TOKENS,
            temperature=BLOG_POST_TEMPERATURE
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error generating blog post: {e}")
        return None

def generate_threads_post(paper, blog_post_url):
    prompt = f"""Create a short, engaging post for Threads (max {THREADS_MAX_CHARS} characters) about this scientific paper:
    Title: {paper.title}
    
    Include a brief highlight of the research and its potential impact. 
    Do not include any hashtags or 'Read more' statements.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that creates engaging social media posts about scientific papers."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=100,
            temperature=BLOG_POST_TEMPERATURE
        )
        threads_text = response.choices[0].message.content.strip().replace(":", "")
        
        full_post = f"{threads_text}\n\n{THREADS_HASHTAGS}\n\nRead more: {blog_post_url}"
        
        if len(full_post) > 500:
            available_chars = 500 - len(THREADS_HASHTAGS) - len(blog_post_url) - 15
            truncated_text = threads_text[:available_chars-3] + "..."
            full_post = f"{truncated_text}\n\n{THREADS_HASHTAGS}\n\nRead more: {blog_post_url}"
        
        return full_post
    except Exception as e:
        print(f"Error generating Threads post: {e}")
        return None
   
def generate_ai_image(paper, threads_post):
    """
    Generate an AI image using DALL-E 3 based on the research paper
    """
    # Create a more specific prompt based on the paper's content
    # Extract key concepts from the paper title and abstract
    title_words = paper.title.lower()
    
    # Determine the research domain for better image prompts
    if any(word in title_words for word in ['neural', 'network', 'deep', 'learning', 'ai', 'artificial']):
        domain = "neural networks and AI"
        visual_style = "futuristic digital networks with glowing nodes and connections"
    elif any(word in title_words for word in ['computer', 'vision', 'image', 'visual']):
        domain = "computer vision"
        visual_style = "digital image processing with geometric patterns and visual data"
    elif any(word in title_words for word in ['nlp', 'language', 'text', 'linguistic']):
        domain = "natural language processing"
        visual_style = "flowing text and language symbols transforming into digital patterns"
    elif any(word in title_words for word in ['robot', 'autonomous', 'control']):
        domain = "robotics"
        visual_style = "sleek robotic elements and autonomous systems"
    elif any(word in title_words for word in ['data', 'analysis', 'mining']):
        domain = "data science"
        visual_style = "abstract data visualizations and flowing information streams"
    else:
        domain = "artificial intelligence research"
        visual_style = "abstract technological concepts with clean, modern design"
    
    # Create a sophisticated prompt
    prompt = f"""
    Create a modern, professional illustration representing {domain} research. 
    The image should feature {visual_style}, using a clean and sophisticated 
    color palette with blues, purples, and subtle gradients. The style should be 
    minimalist yet engaging, suitable for a technology blog. Avoid text, people, 
    or specific logos. Focus on abstract technological concepts that convey 
    innovation and scientific progress.
    """
    
    try:
        print(f"Generating DALL-E 3 image for: {paper.title[:50]}...")
        
        response = client.images.generate(
            model=IMAGE_MODEL,
            prompt=prompt,
            n=1,
            size=IMAGE_SIZE,
            quality=IMAGE_QUALITY,
            style=IMAGE_STYLE
        )
        
        image_url = response.data[0].url
        print("✅ DALL-E 3 image generated successfully!")
        
        # Display the image in the notebook (if running in Jupyter)
        try:
            from IPython.display import display, Image
            display(Image(url=image_url))
        except ImportError:
            print(f"Image URL: {image_url}")
        
        return image_url
        
    except Exception as e:
        print(f"❌ Error generating DALL-E 3 image: {e}")
        
        # Fallback: try with a simpler prompt
        try:
            print("Trying with simplified prompt...")
            simple_prompt = f"A clean, modern illustration representing {domain}, minimalist style, blue and purple gradient background"
            
            response = client.images.generate(
                model=IMAGE_MODEL,
                prompt=simple_prompt,
                n=1,
                size=IMAGE_SIZE,
                quality=IMAGE_QUALITY
            )
            
            image_url = response.data[0].url
            print("✅ Fallback image generated successfully!")
            return image_url
            
        except Exception as e2:
            print(f"❌ Fallback also failed: {e2}")
            return None

def download_and_save_image(image_url, paper_short_id, date):
    """
    Download the AI image and save it to GitHub repository
    This prevents the image from expiring
    """
    import requests
    import base64
    
    try:
        # Download the image
        print("Downloading image...")
        img_response = requests.get(image_url)
        img_response.raise_for_status()
        
        # Create filename
        image_filename = f"assets/images/{date}-{paper_short_id}.png"
        
        # Encode image for GitHub API
        encoded_image = base64.b64encode(img_response.content).decode("utf-8")
        
        # Upload to GitHub
        url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{image_filename}"
        headers = {
            "Authorization": f"token {GITHUB_TOKEN}",
            "Accept": "application/vnd.github.v3+json"
        }
        
        data = {
            "message": f"Add image for blog post {paper_short_id}",
            "content": encoded_image
        }
        
        response = requests.put(url, headers=headers, json=data)
        
        if response.status_code == 201:
            # Return the GitHub-hosted image URL
            github_image_url = f"https://{GITHUB_PAGES_SITE}/{image_filename}"
            print(f"✅ Image saved to GitHub: {github_image_url}")
            
            # Wait for GitHub Pages to deploy the image
            print(f"⏳ Waiting for GitHub Pages to deploy image ({GITHUB_PAGES_IMAGE_WAIT} seconds)...")
            time.sleep(GITHUB_PAGES_IMAGE_WAIT)
            
            # Test if the image is accessible
            try:
                test_response = requests.head(github_image_url, timeout=10)
                if test_response.status_code == 200:
                    print("✅ GitHub Pages image is accessible!")
                    return github_image_url
                else:
                    print(f"⚠️ GitHub Pages image not ready (status: {test_response.status_code})")
                    print("📎 Using original OpenAI URL as fallback")
                    return image_url
            except Exception as e:
                print(f"⚠️ Cannot verify GitHub Pages image accessibility: {e}")
                print("📎 Using original OpenAI URL as fallback")
                return image_url
                
        else:
            print(f"❌ Failed to save image to GitHub: {response.status_code}")
            print(f"Response: {response.text}")
            return image_url  # Return original URL as fallback
            
    except Exception as e:
        print(f"❌ Error saving image: {e}")
        return image_url  # Return original URL as fallback

def create_github_blog_post(paper, content, date, short_id, image_url):
    # Use consistent short_id for the file name
    file_name = f"{date}-{short_id}.md"
    
    # Generate Harvard reference
    harvard_reference = generate_harvard_reference(paper)
    
    # Fixed: No indentation in front matter
    file_content = f"""---
layout: post
title: "{paper.title}"
date: {date} {datetime.now().strftime('%H:%M:%S +0000')}
categories: [blog, AI, research]
image: {image_url}
---
![AI Generated Image]({image_url})

{content}

## Original Research Paper
For more details, please refer to the original research paper:
[{paper.title}]({paper.entry_id})

## Reference
{harvard_reference}
"""
    
    encoded_content = base64.b64encode(file_content.encode("utf-8")).decode("utf-8")
    url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/_posts/{file_name}"
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }
    
    # Check if file already exists
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        print(f"Blog post already exists: {file_name}")
        return False, ""

    # File doesn't exist, create new file
    data = {
        "message": f"Add new blog post: {paper.title}",
        "content": encoded_content
    }

    response = requests.put(url, headers=headers, json=data)
    if response.status_code != 201:
        print(f"GitHub API Error: {response.status_code}")
        print(f"Response content: {response.text}")
        return False, ""
    
    # Construct the URL based on the file name
    post_url = f"https://{GITHUB_PAGES_SITE}/{date.replace('-', '/')}/{short_id}/"
    return True, post_url
   

def check_existing_post(short_id, date):
    file_name = f"{date}-{short_id}.md"
    url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/_posts/{file_name}"
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }
    response = requests.get(url, headers=headers)
    return response.status_code == 200

def create_media_container(access_token, user_id, text, image_url):
    url = f"https://graph.threads.net/v1.0/{user_id}/threads"
    
    params = {
        "media_type": "IMAGE",
        "image_url": image_url,
        "text": text,
        "access_token": access_token
    }
    
    try:
        response = requests.post(url, params=params)
        response.raise_for_status()
        print(f"Create Media Container Status Code: {response.status_code}")
        return response.json()
    except requests.RequestException as e:
        print(f"Error creating media container: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Response content: {e.response.text}")
        return None

def publish_thread(access_token, user_id, creation_id):
    url = f"https://graph.threads.net/v1.0/{user_id}/threads_publish"
    
    params = {
        "creation_id": creation_id,
        "access_token": access_token
    }
    
    try:
        response = requests.post(url, params=params)
        response.raise_for_status()
        print(f"Publish Thread Status Code: {response.status_code}")
        return response.json()
    except requests.RequestException as e:
        print(f"Error publishing thread: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Response content: {e.response.text}")
        return None

def post_to_threads(text, image_url, access_token, user_id, initial_wait=30, max_retries=3):
    for attempt in range(max_retries):
        try:
            # Step 1: Create media container
            container = create_media_container(access_token, user_id, text, image_url)
            if container is None or 'id' not in container:
                print("Failed to create media container.")
                return False

            container_id = container['id']
            print(f"Media container created with ID: {container_id}")

            # Wait before publishing
            print(f"Waiting {initial_wait} seconds before publishing...")
            time.sleep(initial_wait)

            # Step 2: Publish the thread
            publish_result = publish_thread(access_token, user_id, container_id)
            if publish_result is None or 'id' not in publish_result:
                print("Failed to publish thread.")
                return False

            print(f"Successfully posted to Threads with ID: {publish_result['id']}")
            return True

        except Exception as e:
            print(f"Error posting to Threads: {e}")
            
            if attempt < max_retries - 1:
                print(f"Retrying in {initial_wait} seconds...")
                time.sleep(initial_wait)
            else:
                print("Max retries reached. Failed to post to Threads.")
                return False

    return False

def main():
    try:
        if VERBOSE_OUTPUT:
            print("🚀 Starting automated blog creation process...")
            print(f"📝 Configuration: {MAX_PAPERS_TO_PROCESS} papers, {DAYS_BACK_TO_SEARCH} days back")
            print(f"🔍 Categories: {', '.join(ARXIV_CATEGORIES)}")
        
        papers = fetch_latest_papers()
        if not papers:
            print(f"No recent papers found in categories: {ARXIV_CATEGORIES}")
            return

        processed_count = 0
        for paper in papers:
            display(Markdown(f"## Processing: {paper.title}"))
            
            short_id = hashlib.md5(paper.title.encode()).hexdigest()[:8]
            
            date = datetime.now().strftime("%Y-%m-%d")
            if SKIP_EXISTING_POSTS and check_existing_post(short_id, date):
                print(f"Blog post already exists for: {paper.title}")
                print("Skipping to next paper...")
                continue

            blog_post = generate_blog_post(paper)
            if not blog_post:
                print(f"Failed to generate blog post for: {paper.title}")
                print("Skipping to next paper...")
                continue

            display(Markdown(f"### Original Paper: [{paper.entry_id}]({paper.entry_id})"))
            display(Markdown(blog_post))
            
            # Generate a temporary post URL
            temp_post_url = f"https://{GITHUB_PAGES_SITE}/{date.replace('-', '/')}/{short_id}/"
            
            # Generate Threads post first
            threads_post = generate_threads_post(paper, temp_post_url)
            if not threads_post:
                print("Failed to generate Threads post. Skipping to next paper...")
                continue
            
            # Generate image based on paper content
            image_url = generate_ai_image(paper, threads_post)
            if not image_url:
                print("Failed to generate AI image. Skipping to next paper...")
                continue
            
            # Download and save the image to prevent expiration
            if SAVE_IMAGES_TO_GITHUB:
                image_url = download_and_save_image(image_url, short_id, date)
            
            # Create GitHub blog post with the generated image
            success, post_url = create_github_blog_post(paper, blog_post, date, short_id, image_url)
            
            if success:
                print(f"Successfully created blog post on GitHub: {post_url}")
                
                # Add delay to allow for link preview generation
                print(f"Waiting {LINK_PREVIEW_WAIT} seconds for link preview generation...")
                time.sleep(LINK_PREVIEW_WAIT)
                
                # Update the Threads post with the correct URL if it changed
                if post_url != temp_post_url:
                    threads_post = threads_post.replace(temp_post_url, post_url)
                
                display(Markdown(f"### Threads Post:\n{threads_post}"))
                if post_to_threads(threads_post, image_url, THREADS_ACCESS_TOKEN, THREADS_USER_ID, THREADS_WAIT_TIME, THREADS_MAX_RETRIES):
                    print("Successfully posted to Threads with image!")
                else:
                    print("Failed to post to Threads.")
                
                processed_count += 1
            else:
                print("Failed to create blog post on GitHub.")
        
        if VERBOSE_OUTPUT:
            print(f"\n✅ Processing complete! Created {processed_count} new blog posts.")

    except Exception as e:
        print(f"An error occurred: {e}")
        raise

if __name__ == "__main__":
    main()

🚀 Starting automated blog creation process...
📝 Configuration: 5 papers, 7 days back
🔍 Categories: cs.AI, cs.LG, cs.CL, cs.CV, stat.ML
📊 Fetched 5 papers, 5 from last 7 days


## Processing: HunyuanVideo-Avatar: High-Fidelity Audio-Driven Human Animation for Multiple Characters

### Original Paper: [http://arxiv.org/abs/2505.20156v1](http://arxiv.org/abs/2505.20156v1)

Are you ready to witness the future of audio-driven human animation? A groundbreaking study by Yi Chen and team introduces HunyuanVideo-Avatar, a cutting-edge model that brings multiple characters to life in high-fidelity dialogue videos.

So, what does all this scientific jargon actually mean? Let's break it down. In simple terms, HunyuanVideo-Avatar is like a magician that can create dynamic, emotionally expressive avatars that talk and move realistically on screen. Imagine watching a movie where the characters not only speak but also convey their emotions through their facial expressions and body language with incredible accuracy.

One of the key findings of this study is the development of a new technology called the multimodal diffusion transformer (MM-DiT), which allows for the seamless integration of audio and visual cues to generate lifelike animations. By using a character image injection module, the model ensures that each character maintains its unique traits consistently throughout the video, making the animations truly believable.

But why is this research important beyond the realm of entertainment? The implications are vast and exciting. Imagine using this technology in virtual reality experiences to create interactive characters that respond to your voice commands and emotions in real-time. In education, it could revolutionize online learning by providing engaging and personalized avatars for virtual instructors. In the world of gaming, imagine playing a game where the characters not only look realistic but also react emotionally to your gameplay decisions.

The possibilities are endless, and HunyuanVideo-Avatar opens up a whole new world of opportunities for immersive storytelling and interactive experiences. This study not only pushes the boundaries of audio-driven animation but also paves the way for future innovations in artificial intelligence and human-computer interaction.

In a nutshell, HunyuanVideo-Avatar is not just a scientific breakthrough—it's a glimpse into a future where technology blurs the line between reality and imagination. Get ready to be amazed by the magic of high-fidelity audio-driven human animation and step into a world where your favorite characters come to life like never before.

Generating DALL-E 3 image for: HunyuanVideo-Avatar: High-Fidelity Audio-Driven Hu...
✅ DALL-E 3 image generated successfully!


...

Are you tired of complex neural network designs that promise improved performance but come at the cost of higher compute time? Well, a recent scientific paper by Ilai Reshef and Nadav Dym might have just the answer for you! In their study titled "On the (Non) Injectivity of Piecewise Linear Janossy Pooling," the researchers dive deep into the world of multiset functions and shed light on an intriguing discovery.

So, what exactly did Reshef and Dym uncover in their research? Let's break it down into simpler terms. Multiset functions play a crucial role in the development of neural networks for handling multisets and graphs. These functions map multisets (collections of items where duplicates are allowed) to vectors, providing a unique representation for each multiset. One key property desired in these mappings is injectivity, which ensures that each multiset is uniquely represented by a vector.

In their study, the researchers focused on a specific type of multiset function known as k-ary Janossy pooling, a popular choice in the realm of multiset models. They set out to investigate whether piecewise linear Janossy pooling functions could achieve injectivity. The result? A surprising finding that no piecewise linear Janossy pooling function can be injective.

But fear not, as there's a silver lining to this discovery! Reshef and Dym also demonstrated that even simple deep-sets models, when applied to multisets without multiplicities, can still achieve the desired properties of injectivity and bi-Lipschitzness. This opens up the possibility of using more straightforward multiset functions without compromising on the integrity of the representations.

So, what does this mean for the real world? Well, the implications of this research could lead to more efficient neural network designs that strike a balance between performance and computational resources. By exploring simpler alternatives to complex multiset functions, researchers and developers may find ways to streamline their models without sacrificing accuracy.

In conclusion, Reshef and Dym's paper challenges the status quo in multiset function design and paves the way for new possibilities in the realm of neural networks. Who knew that the quest for simplicity could uncover such fascinating insights in the world of computational science?

Generating DALL-E 3 image for: On the (Non) Injectivity of Piecewise Linear Janos...
✅ DALL-E 3 image generated successfully!


✅ Image saved to GitHub: https://porkpy.github.io/research_blogger/assets/images/2025-05-27-6e617040.png
Successfully created blog post on GitHub: https://porkpy.github.io/research_blogger/2025/05/27/6e617040/
Waiting 30 seconds for link preview generation...


### Threads Post:
🔬 New research on Piecewise Linear Janossy Pooling reveals surprising insights on its injectivity! This study challenges traditional assumptions and opens up exciting possibilities for improved data processing methods. Stay ahead of the curve with the latest findings in computational science! 🌟 #research #science #dataanalysis

#AI #ArtificialIntelligence #MachineLearning #DataScience #Latest #Research #Arxiv #OpenAI

Read more: https://porkpy.github.io/research_blogger/2025/05/27/6e617040/

Error creating media container: 400 Client Error: Bad Request for url: https://graph.threads.net/v1.0/7610082715761011/threads?media_type=IMAGE&image_url=https%3A%2F%2Fporkpy.github.io%2Fresearch_blogger%2Fassets%2Fimages%2F2025-05-27-6e617040.png&text=%F0%9F%94%AC+New+research+on+Piecewise+Linear+Janossy+Pooling+reveals+surprising+insights+on+its+injectivity%21+This+study+challenges+traditional+assumptions+and+opens+up+exciting+possibilities+for+improved+data+processing+methods.+Stay+ahead+of+the+curve+with+the+latest+findings+in+computational+science%21+%F0%9F%8C%9F+%23research+%23science+%23dataanalysis%0A%0A%23AI+%23ArtificialIntelligence+%23MachineLearning+%23DataScience+%23Latest+%23Research+%23Arxiv+%23OpenAI%0A%0ARead+more%3A+https%3A%2F%2Fporkpy.github.io%2Fresearch_blogger%2F2025%2F05%2F27%2F6e617040%2F&access_token=THQVFUWGI1SWVlUmlaamlVQVdId3FqVlQ3aWtYNzgzeE4zbHg1bDB3SS1CZAjFBcGdVTXdmaks3ZAEVSSUZAjcExYR3VMTG5jSlZAKeWtkYmFTNUJ5QWJLdnRJMlZAMSDZA3NzJmTzZAjZA3lETGY0ZAzJGbHRmeEh